<a href="https://colab.research.google.com/github/pninav/Cycle_Gan/blob/main/Cycle_GAN_Eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mounting to google-drive

In [ ]:
from google.colab import drive
import os
drive.mount('/content/Pnina/')
os.chdir('/content/Pnina/MyDrive/')

Mounted at /content/Pnina/


Cloning CycleGan project (creates a local directory in google drive, initializes a GitHub directory inside it), and installing requirements. 

In [ ]:
!rm -rf /content/Pnina/MyDrive/Cycle_Gan
!git clone https://github.com/pninav/Cycle_Gan.git
os.chdir('/content/Pnina/MyDrive/Cycle_Gan')
!pip install -r requirements.txt  

Cloning into 'Cycle_Gan'...
remote: Enumerating objects: 96, done.
remote: Counting objects: 100% (96/96), done.
remote: Compressing objects: 100% (94/94), done.
remote: Total 96 (delta 52), reused 8 (delta 0), pack-reused 0
Unpacking objects: 100% (96/96), done.
     |████████████████████████████████| 676 kB 11.5 MB/s 
     |████████████████████████████████| 195 kB 63.7 MB/s 
     |████████████████████████████████| 108 kB 58.2 MB/s 
     |████████████████████████████████| 26.9 MB 1.5 MB/s 
     |████████████████████████████████| 68 kB 7.8 MB/s 
  Created wheel for visdom: filename=visdom-0.1.8.9-py3-none-any.whl size=655249 sha256=16c8df26437a5aea9dc85c07f2ad0525671d97526277fa0425c6685d13cda7fb
  Stored in directory: /root/.cache/pip/wheels/2d/d1/9b/cde923274eac9cbb6ff0d8c7c72fe30a3da9095a38fd50bbf1
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7410 sha256=ee8ef82f98da7f26ff59744083f7812899fdce15fd405c6ec5168bcd03a27eb3
  Stored in directory: /root/.cache/pip

Download Kaggle dataset using Kaggle API.

In [ ]:
import os
os.chdir('/content/Pnina/MyDrive/Cycle_Gan/')
os.environ['KAGGLE_CONFIG_DIR'] = '/content/Pnina/MyDrive/Cycle_Gan/kaggle/'
!mkdir -p kaggle_dataset
os.chdir('/content/Pnina/MyDrive/Cycle_Gan/kaggle_dataset')
!kaggle competitions download -c gan-getting-started 

  0% 0.00/1.88M [00:00<?, ?B/s]
100% 1.88M/1.88M [00:00<00:00, 62.3MB/s]
  0% 0.00/1.87M [00:00<?, ?B/s]
100% 1.87M/1.87M [00:00<00:00, 127MB/s]
  0% 0.00/2.12M [00:00<?, ?B/s]
100% 2.12M/2.12M [00:00<00:00, 70.0MB/s]
  0% 0.00/1.93M [00:00<?, ?B/s]
100% 1.93M/1.93M [00:00<00:00, 131MB/s]
  0% 0.00/1.94M [00:00<?, ?B/s]
100% 1.94M/1.94M [00:00<00:00, 64.4MB/s]
  0% 0.00/9.16k [00:00<?, ?B/s]
100% 9.16k/9.16k [00:00<00:00, 2.90MB/s]
  0% 0.00/17.9k [00:00<?, ?B/s]
100% 17.9k/17.9k [00:00<00:00, 2.51MB/s]
  0% 0.00/18.0k [00:00<?, ?B/s]
100% 18.0k/18.0k [00:00<00:00, 2.56MB/s]
  0% 0.00/15.0k [00:00<?, ?B/s]
100% 15.0k/15.0k [00:00<00:00, 2.11MB/s]
  0% 0.00/25.4k [00:00<?, ?B/s]
100% 25.4k/25.4k [00:00<00:00, 3.54MB/s]
  0% 0.00/17.6k [00:00<?, ?B/s]
100% 17.6k/17.6k [00:00<00:00, 2.41MB/s]
  0% 0.00/23.4k [00:00<?, ?B/s]
100% 23.4k/23.4k [00:00<00:00, 3.33MB/s]
  0% 0.00/19.7k [00:00<?, ?B/s]
100% 19.7k/19.7k [00:00<00:00, 5.84MB/s]
  0% 0.00/11.2k [00:00<?, ?B/s]
100% 11.2k/11.2k [00:

Create Cycle_Gan folder (the GitHub files will be transferred there)

In [ ]:
os.chdir('/content/Pnina/MyDrive/Cycle_Gan/')


Run the train.py notebook - Train the CycleGAN model.

In [ ]:
!python train.py --cuda --eval

2021-07-31 19:35:27.104949: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
Namespace(batchSize=1, cuda=True, dataroot='/content/Pnina/MyDrive/Cycle_Gan/kaggle_dataset/', decay_epoch=100, epoch=0, eval=True, input_nc=3, kaggle=False, lambda_cycle=10, lr=0.0002, n_cpu=8, n_epochs=200, output_nc=3, size=256)
/content/Pnina/MyDrive/Cycle_Gan/utils.py:130: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  torch.nn.init.normal(m.weight.data, 0.0, 0.02)
/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:281: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "
2021-07-31 19:35:37.618290: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-07-31 19:35:37.61869

Create output & run folders
Download the training weights

In [ ]:
import os
os.chdir('/content/Pnina/MyDrive/Cycle_Gan/')
!mkdir -p output
os.chdir('/content/Pnina/MyDrive/Cycle_Gan/output')
!gdown --id 1PLazQWL6YmA8l-W-gEUib2aGSKfRyWgH
!gdown --id 1d7on629bJcAsUvi8Pd5j-I95UhrxxN30
os.chdir('/content/Pnina/MyDrive/Cycle_Gan/')
!mkdir -p runs 
os.chdir('/content/Pnina/MyDrive/Cycle_Gan/runs')
!mkdir -p best 
os.chdir('/content/Pnina/MyDrive/Cycle_Gan/runs/best')
!gdown --id 1rhrDBBOostvxDr39MMk3kY7tHzDi8Ptf



Downloading...
From: https://drive.google.com/uc?id=1PLazQWL6YmA8l-W-gEUib2aGSKfRyWgH
To: /content/Pnina/My Drive/Cycle_Gan/output/netG_A2B.pth
45.5MB [00:00, 115MB/s] 
Downloading...
From: https://drive.google.com/uc?id=1d7on629bJcAsUvi8Pd5j-I95UhrxxN30
To: /content/Pnina/My Drive/Cycle_Gan/output/netG_B2A.pth
45.5MB [00:00, 145MB/s]
Permission denied: https://drive.google.com/uc?id=1rhrDBBOostvxDr39MMk3kY7tHzDi8Ptf
Maybe you need to change permission over 'Anyone with the link'?


Run the test.py notebook While displaying a progress bar for each iteration

In [ ]:
os.chdir('/content/Pnina/MyDrive/Cycle_Gan/')
!python test.py --cuda

Generated images 0161 of 7038Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/PIL/ImageFile.py", line 496, in _save
    fh = fp.fileno()
AttributeError: '_idat' object has no attribute 'fileno'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "test.py", line 96, in <module>
    save_image(fake_B, 'output/B/%04d.png' % (i+1))
  File "/usr/local/lib/python3.7/dist-packages/torch/autograd/grad_mode.py", line 28, in decorate_context
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/torchvision/utils.py", line 136, in save_image
    im.save(fp, format=format)
  File "/usr/local/lib/python3.7/dist-packages/PIL/Image.py", line 2134, in save
    save_handler(self, fp, filename)
  File "/usr/local/lib/python3.7/dist-packages/PIL/PngImagePlugin.py", line 1291, in _save
    ImageFile._save(im, _idat(fp, chunk), [("zip", (0, 0) + im.size, 0, rawmode)])
  File "/usr/local/li

View Images

In [ ]:
from google.colab.patches import cv2_imshow
from glob import glob
import cv2

for file in glob('/content/Pnina/MyDrive/Cycle_Gan/output/B_reduced2/*.*'):
  img = cv2.imread(file, cv2.IMREAD_UNCHANGED)
  cv2_imshow(img) 

Create an output file (zip file) containing approximately 7000 Monet-style images (the input photos in Monet style)

---



In [ ]:
!mkdir /content/Pnina/MyDrive/Cycle_Gan/output/submission
!zip -r  /content/Pnina/MyDrive/Cycle_Gan/output/submission/Pnina.zip /content/Pnina/MyDrive/Cycle_Gan/output/B

Streaming output truncated to the last 5000 lines.
  adding: content/Pnina/MyDrive/Cycle_Gan/output/B/2039.png (deflated 0%)
  adding: content/Pnina/MyDrive/Cycle_Gan/output/B/2040.png (deflated 0%)
  adding: content/Pnina/MyDrive/Cycle_Gan/output/B/2041.png (deflated 0%)
  adding: content/Pnina/MyDrive/Cycle_Gan/output/B/2042.png (deflated 0%)
  adding: content/Pnina/MyDrive/Cycle_Gan/output/B/2043.png (deflated 0%)
  adding: content/Pnina/MyDrive/Cycle_Gan/output/B/2044.png (deflated 0%)
  adding: content/Pnina/MyDrive/Cycle_Gan/output/B/2045.png (deflated 0%)
  adding: content/Pnina/MyDrive/Cycle_Gan/output/B/2046.png (deflated 0%)
  adding: content/Pnina/MyDrive/Cycle_Gan/output/B/2047.png (deflated 0%)
  adding: content/Pnina/MyDrive/Cycle_Gan/output/B/2048.png (deflated 0%)
  adding: content/Pnina/MyDrive/Cycle_Gan/output/B/2049.png (deflated 0%)
  adding: content/Pnina/MyDrive/Cycle_Gan/output/B/2050.png (deflated 0%)
  adding: content/Pnina/MyDrive/Cycle_Gan/output/B/2051.png (

Using TensorBoard - view dashboards such as scalars, graphs, histograms, and others results.

In [ ]:
%load_ext tensorboard
%tensorboard --logdir=/content/Pnina/MyDrive/Cycle_Gan/runs  --port 6006


Downloaded a pre-trained model on ImageNet of Inception v3 from http://download.tensorflow.org/models/image/imagenet/inception-2015-12-05.tgz  


In [ ]:
!wget http://download.tensorflow.org/models/image/imagenet/inception-2015-12-05.tgz

--2021-07-27 12:42:26--  http://download.tensorflow.org/models/image/imagenet/inception-2015-12-05.tgz
Resolving download.tensorflow.org (download.tensorflow.org)... 172.217.9.208, 2607:f8b0:4004:806::2010
Connecting to download.tensorflow.org (download.tensorflow.org)|172.217.9.208|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 88931400 (85M) [application/x-compressed-tar]
Saving to: ‘inception-2015-12-05.tgz’

inception-2015-12-0 100%[===================>]  84.81M  73.7MB/s    in 1.2s    

2021-07-27 12:42:27 (73.7 MB/s) - ‘inception-2015-12-05.tgz’ saved [88931400/88931400]



Extract the zip of the inception, especially the weights (classify_image_graph_def.pb)

In [ ]:
os.chdir('/content/Pnina/MyDrive/Cycle_Gan')
!tar zxvf /content/Pnina/MyDrive/Cycle_Gan/inception-2015-12-05.tgz

classify_image_graph_def.pb
cropped_panda.jpg
imagenet_2012_challenge_label_map_proto.pbtxt
imagenet_synset_to_human_label_map.txt
LICENSE


In [ ]:
!pip install tensorflow==1.15
!pip3 install protobuf==3.6.0


     |████████████████████████████████| 412.3 MB 25 kB/s 
     |████████████████████████████████| 50 kB 7.7 MB/s 
     |████████████████████████████████| 3.8 MB 43.6 MB/s 
     |████████████████████████████████| 503 kB 72.9 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7553 sha256=0828fbd979d7d02fde846dd038dc462be43107f89fca790ef26923d0dcee466a
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.5.0
    Uninstalling tensorflow-estimator-2.5.0:
      Successfully uninstalled tensorflow-estimator-2.5.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.5.0
    Uninstalling tensorboard-2.5.0:
      Successfully uninstalled tensorboard-2.5.0
  Attempting uninstall: gast
    Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Succes

     |████████████████████████████████| 390 kB 33.3 MB/s 
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.17.3
    Uninstalling protobuf-3.17.3:
      Successfully uninstalled protobuf-3.17.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 1.15.0 requires protobuf>=3.6.1, but you have protobuf 3.6.0 which is incompatible.
tensorflow-metadata 1.1.0 requires protobuf<4,>=3.13, but you have protobuf 3.6.0 which is incompatible.
tensorflow-hub 0.12.0 requires protobuf>=3.8.0, but you have protobuf 3.6.0 which is incompatible.
tensorflow-datasets 4.0.1 requires protobuf>=3.6.1, but you have protobuf 3.6.0 which is incompatible.
kapre 0.3.5 requires tensorflow>=2.0.0, but you have tensorflow 1.15.0 which is incompatible.
googleapis-common-protos 1.53.0 requires protobuf>=3.12.0, but you have protobuf 3.6.0 which is incompatible

You must restart the runtime in order to use newly installed versions
A button will be displayed to confirm a restart to upload an advanced version of TensorFlow.


Initialize parameters for MiFID:

  I.	MODEL_PATH: inception weights

  II.	TRAIN_DIR:  thirty reduced Monet images we selected for training
  
  III.	OUT_DIR:  thirty Monet style output images


In [ ]:
import os
from pathlib import Path
import shutil

import numpy as np
import torch
from torch import nn, optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
MODEL_PATH = '/content/Pnina/MyDrive/Cycle_Gan/classify_image_graph_def.pb'
TRAIN_DIR = Path('/content/Pnina/MyDrive/Cycle_Gan/kaggle_dataset/monet_reduced' )
OUT_DIR = Path('/content/Pnina/MyDrive/Cycle_Gan/output/B_reduced')
OUT_DIR.mkdir(exist_ok=True)


MiFID - evaluation method

In [ ]:
from dataclasses import dataclass
from pathlib import Path
import warnings

import numpy as np
from PIL import Image
from scipy import linalg
import tensorflow as tf


class KernelEvalException(Exception):
    pass


@dataclass
class MiFIDEvaluator(object):
    model_path: str
    train_images_path: str
    feature_path: str = None
    imsize: int = 256
    output_layer: str = 'Pretrained_Net/pool_3:0'
    input_layer: str = 'Pretrained_Net/ExpandDims:0'
    output_shape: int = 2048
    cosine_distance_eps: float = 0.1
    batch_size: int = 1
    fid_epsilon: float = 1e-14
    
    def __post_init__(self):
        tf.reset_default_graph()
        self.create_model_graph()
        with tf.Session() as sess:
            if self.feature_path is None:
                self.mu2, self.sigma2, self.features2 = self._handle_path_memorization(
                    self.train_images_path, sess, is_checksize=False, is_check_png=False)
            else:
                with np.load(self.feature_path) as f:
                    self.mu2, self.sigma2, self.features2 = f['m'], f['s'], f['features']
    
    def create_model_graph(self):
        with tf.gfile.FastGFile(self.model_path, 'rb') as f:
            graph_def = tf.GraphDef()
            graph_def.ParseFromString(f.read())
            _ = tf.import_graph_def(graph_def, name='Pretrained_Net')
            
    def img_read_checks(self, filename, is_checksize=False, is_check_png=False,check_imsize = 256):
        im = Image.open(str(filename)).convert('RGB')
        if is_checksize and im.size != (self.imsize, self.imsize):
            raise KernelEvalException(f'The images are not of size {check_imsize}')
    

        if self.imsize is None:
            return im
        else:
            return im.resize((self.imsize, self.imsize), Image.ANTIALIAS)
        
    def _get_model_layer(self, sess):
        layer = sess.graph.get_tensor_by_name(self.output_layer)
        ops = layer.graph.get_operations()
        for op_idx, op in enumerate(ops):
            for o in op.outputs:
                shape = o.get_shape()
                if shape._dims != []:
                    shape = [s.value for s in shape]
                    new_shape = []
                    for j, s in enumerate(shape):
                        if s == 1 and j == 0:
                            new_shape.append(None)
                        else:
                            new_shape.append(s)
                    o.__dict__['_shape_val'] = tf.TensorShape(new_shape)
        return layer
        
    def get_activations(self, images, sess):
        inception_layer = self._get_model_layer(sess)
        n_images = images.shape[0]
        if self.batch_size > n_images:
            warnings.warn('batch size is bigger than the data size. setting batch size to data size')
            self.batch_size = n_images
        print("NUmber of Images",n_images)
        n_batches = n_images // self.batch_size + 1
        pred_arr = np.empty((n_images, self.output_shape))
        for i in range(n_batches):
            start = i * self.batch_size
            if start + self.batch_size < n_images:
                end = start + self.batch_size
            else:
                end = n_images

            batch = images[start:end]
            pred = sess.run(inception_layer, {self.input_layer: batch})
            pred_arr[start:end] = pred.reshape(-1, self.output_shape)
        return pred_arr
        
    def calculate_activation_statistics(self, images, sess):
        act = self.get_activations(images, sess)
        mu = np.mean(act, axis=0)
        sigma = np.cov(act, rowvar=False)
        return mu, sigma, act
            
    def _handle_path_memorization(self, path, sess, is_checksize, is_check_png):
        path = Path(path)
        files = list(path.glob('*.jpg')) + list(path.glob('*.png'))

        # In production we don't resize input images. This is just for demo purpose. 
        x = np.array([np.array(self.img_read_checks(fn, is_checksize, is_check_png)) for fn in files])
        m, s, features = self.calculate_activation_statistics(x, sess)
        del x
        return m, s, features
    
    def calculate_frechet_distance(self, mu1, sigma1):
        mu1 = np.atleast_1d(mu1)
        mu2 = np.atleast_1d(self.mu2)
        sigma1 = np.atleast_2d(sigma1)
        sigma2 = np.atleast_2d(self.sigma2)

        assert mu1.shape == mu2.shape, 'Training and test mean vectors have different lengths'
        assert sigma1.shape == sigma2.shape, 'Training and test covariances have different dimensions'

        # product might be almost singular
        covmean, _ = linalg.sqrtm(sigma1.dot(sigma2), disp=False)
        if not np.isfinite(covmean).all():
            msg = f'fid calculation produces singular product; adding {self.eps} to diagonal of cov estimates'
            warnings.warn(msg)
            offset = np.eye(sigma1.shape[0]) * self.eps
            covmean = linalg.sqrtm((sigma1 + offset).dot(sigma2 + offset))
            
        # numerical error might give slight imaginary component
        if np.iscomplexobj(covmean):
            if not np.allclose(np.diagonal(covmean).imag, 0, atol=1e-3):
                m = np.max(np.abs(covmean.imag))
                raise ValueError(f'Imaginary component {m}')
            covmean = covmean.real
        tr_covmean = np.trace(covmean)
        return (mu1 - mu2).dot(mu1 - mu2) + np.trace(sigma1) + np.trace(sigma2) - 2 * tr_covmean
    
    @staticmethod
    def normalize_rows(x):
        return np.nan_to_num(x / np.linalg.norm(x, ord=2, axis=1, keepdims=True))
    
    def cosine_distance(self, features1):
        features1_nozero = features1[np.sum(features1, axis=1) != 0]
        features2_nozero = self.features2[np.sum(self.features2, axis=1) != 0]
        norm_f1 = self.normalize_rows(features1_nozero)
        norm_f2 = self.normalize_rows(features2_nozero)

        d = 1.0 - np.abs(np.matmul(norm_f1, norm_f2.T))
        mean_min_d = np.mean(np.min(d, axis=1))
        return mean_min_d
            
    def calculate_kid_given_paths(self, user_images_unzipped_path):
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            m1, s1, features1 = self._handle_path_memorization(
                user_images_unzipped_path, sess, is_checksize=True, is_check_png=True)

            fid_value = self.calculate_frechet_distance(m1, s1)
            distance = self.cosine_distance(features1)
            return fid_value, distance
        
    def distance_thresholding(self, d):
        if d < self.cosine_distance_eps:
            return d
        else:
            return 1
        
    def evaluate(self, user_images_unzipped_path):
        fid_value, distance = self.calculate_kid_given_paths(user_images_unzipped_path)
        distance = self.distance_thresholding(distance)
        return fid_value, distance, fid_value / (distance + self.fid_epsilon)

Get MiFID score

In [ ]:
evaluator = MiFIDEvaluator(MODEL_PATH, TRAIN_DIR)
fid_value, distance, mi_fid_score = evaluator.evaluate(OUT_DIR)
print(f'FID: {fid_value:.5f}')
print(f'distance: {distance:.5f}')
print(f'MiFID: {mi_fid_score:.5f}')

NUmber of Images 30
NUmber of Images 30
FID: 275.88140
distance: 1.00000
MiFID: 275.88140
